## Salary Prediction Dataset 

Questo progetto si prepone di risolvere il problema della __Salary Prediction__ in versione *Classification*, quindi di raggruppare i dati in due classi:
- _Salary <= 50k_ Per le persone il cui slario è predetto inferiore a 50k
- _Salary > 50k_ Per l'inverso;

Iniziamo aprendo il dataset dal folder apposito e leggendolo con _pandas_

## Salary Prediction Dataset 

Questo progetto si prepone di risolvere il problema della __Salary Prediction__ in versione *Classification*, quindi di raggruppare i record in due classi:
- _Salary <= 50k_ Per le persone il cui slario è predetto inferiore a 50k
- _Salary > 50k_ Per l'inverso;

Iniziamo aprendo il dataset dal folder apposito e leggendolo con _pandas_

In [ ]:
import pandas as pd
import os

os.chdir("dataset")
df = pd.read_csv("salary.csv")
df.head()

### Data Exploration
Cominciamo vedendo in che stato versa il Dataset cosicchè si possa capire che tipo di operazioni vanno svolte 

In [ ]:
df.columns

In [ ]:
df.info()

Si può notare che tra i tipi delle colonne ci sono principalmente valori _object_, sicuramente servirà un'operazione di *encoding*

In [ ]:
df.describe()

Strane queste colonne capital-gain e capital-loss, meglio dare un'occhiata più specifica

In [ ]:
df['capital-gain'].value_counts()

In [ ]:
df['capital-gain'].value_counts()

Capital-gain e capital-value essere inutili, precisamente, (facendo i conti una sola volta perché il numero di 0 nelle due colonne è lo stesso) si può dire che su 32561 tuple 29849 sono 0.
Infatti impostando la semplice proporzione 
$$
\frac{29849}{32561} \cdot 100 \approx 91.6\%
$$  
Capiamo che addirittura il _*91.6%*_ dei valori all'interno di queste due colonne è 0, questo è abbastanza per rimuovere entrambe le colonne per rendere il dataset più semplice.


Per capire più facilmente quanti siano gli zeri a confronto dei valori accettabili ecco anche una rappresentazione grafica, ma prima un leggero setup di _MathPlotLib_ per avere dei grafici più carini

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
plt.rcParams.update({
    'figure.facecolor': '#1d2021',
    'axes.facecolor': '#201d16',
    'axes.edgecolor': '#ebdbb2',
    'axes.labelcolor': '#ebdbb2',
    'xtick.color': '#ebdbb2',
    'ytick.color': '#ebdbb2',
    'text.color': '#ebdbb2',
    'axes.titlecolor': '#ebdbb2',
    'grid.color': '#3c3836',
})

In [ ]:
plt.figure(figsize=(16, 6)) 
plt.subplot(1, 2, 1)
sns.histplot(df["capital-loss"],bins=100)

plt.subplot(1, 2, 2)
sns.histplot(df["capital-gain"], kde=False,bins=100)

In [ ]:
df.isnull().all()

In [ ]:
for col in df.columns:
    print(f"\nColonna: {col}")
    print(df[col].value_counts())

Tramite l'output di questi ultimi metodi è possibile accorgersi che non sono presenti valori _null_, ma sono presenti molti _missing values_ marchiati con il carattere _?_ nelle colonne:
- WorkClass
- Occupation
- Native Coutry

Inoltre, si può osservare che la colonna _education_ non si differenzia sostanzialmente da _education-num_, se non per il fatto che quest'ultima contiene valori numerici. Considerando che durante il preprocessing andremo ad applicare tecniche di _encoding_ sulle colonne con valori non numerici, è più conveniente mantenere _education-num_. Quest'ultima, infatti, rappresenta già un'ordinamento naturale dei livelli di istruzione, che andrebbe perso utilizzando algoritmi di encoding come il *One Hot Encoding*.

Infine anche la colonna fnlwgt (Final Weight) è inutile perché rappresenta un peso usato nell'analisi demografica dei sondaggi che, nel contesto della classificazione è completamente inutile. 

Ora guardiamo un boxplot delle colonne numeriche, facciamoci un'idea degli _outlier_

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(df[["age","hours-per-week"]])
plt.title("BoxPlot di Age e Hours Per Week")
plt.ylabel("Valore Numerico")
plt.xlabel("Attributo")
plt.tight_layout()
plt.show()

Ci sono dei valori lontani, ma non irrelistici quindi _non serve pulire gli outlier_ 

Infine vediamo qualche grafico per farci un'idea della distribuzione del _dataset_

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x="salary")

plt.title("Distribuzione dei salari")
plt.xlabel("Classe Salariale")
plt.ylabel("Numero di osservazioni")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Da qui si può notare un grande _class imbalance_

## Preprocessing
Cominciamo dalla rimozione delle colonne che erano risultate inutili dalla Data Exploration

In [ ]:
df.drop(["capital-gain", "capital-loss","education","fnlwgt"], axis=1, inplace=True)

Continuando procediamo alla rimozione dei duplicati

In [ ]:
df.drop_duplicates(inplace=True)

Per prima cosa rendiamo più facile le operazioni trasformando i _missing values ' ?'_  in _np.nan_ 

In [ ]:
import numpy as np
df.replace(to_replace=" ?",value=np.nan,inplace=True)

Trasformiamo i _missing values_ nel valore più comune.

In [ ]:
from sklearn.impute import SimpleImputer 

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

Adesso il problema dei _missing values_ è risolto! 

Prima dell'encoding però è importante separare il _target_ dalle _feature_

In [ ]:
x = df.drop(columns=['salary'],axis=1)
y = df['salary']

In [ ]:
columns = ['workclass', 'marital-status', 'occupation','relationship', 'race', 'sex', 'native-country']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first',sparse_output=False)

for column in columns:
    x[column] = encoder.fit_transform(x[[column]])


Separatamente codifichiamo il _target_ con un _label encoder_, visto che _salary_ è un attributo già binario

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

Infine per uniformità dei dati eseguiamo il casting dell'intero contenuto del _Data Frame_ a _float_

In [ ]:
x = x.astype(float)
y = y.astype(float)

Poi eseguiamo la separazione tra _test set_ e _training set_

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Infine per rendere più pratico il funzionamento di alcuni algoritmi di classificazione andiamo a svolgere la _normalizzazione_ dei dati di *X*

Anche se la scala dei valori numerici nel dataset non presenta squilibri eccessivi, per buona pratica si preferisce comunque applicare uno scaling, specialmente in vista dell’utilizzo di algoritmi come KNN e SVM, noti per essere sensibili alla scala delle feature.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_test = scaler.fit_transform(x_test)
x_train = scaler.fit_transform(x_train)

Ora che abbiamo terminato il preprocessing e reso Salary un valore numerico possiamo permetterci di guardare una heatmap che mostri la correlazioni tra gli attributi.

In [ ]:
heat_df = x[['education-num','hours-per-week','age']].copy()
heat_df['salary'] = y

corr = heat_df.corr(numeric_only=True)
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True)
plt.title("Heatmap delle Correlazioni", fontsize=16)
plt.tight_layout() 
plt.show()

## Addestramento dei Modelli

Prima di addestrare i modelli prepariamoci dei metodi che ci permettano di analizzarli:

Due per mostrare dei grafici 

In [ ]:
from sklearn.metrics import confusion_matrix
def matrice_confusione(y_test,y_pred):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix',fontsize = 14, fontweight = 'bold')
    plt.ylabel('Vero')
    plt.xlabel('Predetto')

In [ ]:
def grafico_scatole(risultati,alg):
    sns.barplot(risultati)

    plt.title("Prestazione "+alg, fontsize=14, fontweight='bold')
    plt.ylabel("Valore (%)")
    plt.xlabel("Metrica")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis='y', linestyle='--', alpha=0.4)

Uno che ci calcoli le statistiche in maniera dinamica

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
def statistiche(y_test, y_pred):
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred)
    metrics_df = pd.DataFrame({
        'Precision': np.ceil(precision*100),
        'Recall': np.ceil(recall*100),
        'F1-Score':np.ceil(f1*100),
    }, index=['Classe 0', 'Classe 1'])
    return metrics_df

E uno che unisca tutte le funzioni un uno

In [ ]:
from IPython.display import display
def analizza(y_test,y_pred,alg):
    plt.figure(figsize=(18, 6)) 
    plt.subplot(1, 2, 1)
    matrice_confusione(y_test,y_pred)
    tmp = statistiche(y_test,y_pred)
    display(tmp)
    plt.subplot(1, 2, 2)
    grafico_scatole(tmp,alg)
    results[alg] = tmp
    plt.tight_layout()
    plt.show()

In [ ]:
results = {}

### Decision Tree

Il primo algoritmo che useremo sarà il _Decision tree_, il più semplice e anche il primo studiato, importante è settare la flag *class_weight* come _balanced_ cosìcchè il problema della _class imbalance_ in __y__ sia mitigato, infatti questa flag va regolare il peso delle classi in modo tale che in relazione alla dimensione delle due il loro valore sia bilanciato 

In [ ]:
from sklearn import tree

dec_tree = tree.DecisionTreeClassifier(class_weight= "balanced")
dec_tree.fit(x_train,y_train)

Adesso che il modello è stato addestrato andiamo a vedere come si comporta col _test set_

In [ ]:
y_pred = dec_tree.predict(x_test)

In [ ]:
analizza(y_test,y_pred,"Decision Tree")

Non sono male come risultati, ma il classificatore tende a essere restio nel predire la _classe positiva_ una classifica in base alla media dell'*F1-Score* sulla preditiva_ e quindi per la _classe 0_ abbiamo una bassa *Precision*

### Support Vector Machine 
Ci muoviamo a questa famiglia di algoritmi, _sklearn_ ci fornisce 3 classi per la classificazione con SVM, useremo la versione LinearSVC (*Support Vector Classification*), è molto simile al classico SVC, ma è più veloce, usa come kernel lineare ed è implementato con una libreria più flessibile. 

In [ ]:
from sklearn import svm 
svm_clf = svm.LinearSVC(class_weight = "balanced")
svm_clf.fit(x_train,y_train)

In [ ]:
y_pred = svm_clf.predict(x_test)

In [ ]:
analizza(y_test,y_pred,"SVM")

SVM presenta degli ottimi valori, anche lui predice con poca precisione la *classe 0*, però meglio di Decision Tree

### Nearest Neighbor
Il valore di _k_ utilizzato sarà 9, questo perché scendendo con i valori il modello il modello perdeva di precision, salendo invece diventava troppo sensibile al *rumore*

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=9)
neigh.fit(x_train,y_train)

In [ ]:
y_pred = neigh.predict(x_test)

In [ ]:
analizza(y_test,y_pred,"Nearest Neighbor")

KNN invece guadagna di *Precision*, ma perde di *Recall*, capiamo quindi che molti meno dei reali record $\ge$ 50k sono individuati  

### Percettrone

Ora ci muoviamo all'ANN con il _Percettrone_

In [ ]:
from sklearn.linear_model import Perceptron

ann = Perceptron(max_iter=1000, eta0=0.1, random_state=42)
ann.fit(x_train, y_train)

In [ ]:
y_pred = ann.predict(x_test)

In [ ]:
analizza(y_test,y_pred, "Percettrone")

### Random Forest

Infine diamoci ai classificatori _ensemble_ con *Random Forest*

In [ ]:
from sklearn.ensemble import RandomForestClassifier
ran_forest = RandomForestClassifier(n_estimators=100, class_weight='balanced')
ran_forest.fit(x_train, y_train)

In [ ]:
y_pred = ran_forest.predict(x_test)

In [ ]:
analizza(y_test,y_pred, "Random Forest")

## Risultati Finali

Per prima cosa visualizziamo le statistiche in modo tabulare

In [ ]:
for k in results.keys():
    print(k)
    display(results.get(k))

E poi dei comodi _boxplot_

In [ ]:
plt.figure(figsize=(18, 8)) 

for idx, (alg, df) in enumerate(results.items(), 1):
    plt.subplot(2, 3, idx)
    grafico_scatole(df, alg)

plt.tight_layout()
plt.show()

Ora guardiamo una classifica in base alla media dell'*F1-Score* sulla predizione delle due classi 

In [ ]:
valori = []
for alg,df in results.items():
    valori.append({
        'Modello': alg,
        'Precision (avg)': df['Precision'].mean(),
        'Recall (avg)': df['Recall'].mean(),
        'F1-Score (avg)': df['F1-Score'].mean()
    })

valori = pd.DataFrame()
display(valori.sort_values(by = 'F1-Score (avg)',ascending = False))